# Vehicle Model
## DARPA Assured Micro-Patching Project
## Peter Lobato, Matt Campo, Ben Ettlinger, Subhojeet Mukherjee


This jupyter notebook contains a simulated vehicle model. It pairs with a separate notebook, to be run on a separate kernel, which simulates a vehicle's cruise control module (CCM).

The vehicle model takes in an accelerator pedal position (APP) and outputs vehicle speed in a loop. This runs continuously to simulate a vehicle driving down a road using cruise control in real-time. If the cruise control module is turned off, or the vehicle model stops receiving APP values, it will simulate a coastdown.

The vehicle model takes in APP, via a CAN bus, and outputs incremental changes in vehicle speed also via a CAN bus. 

The vehicle used in this model is a Class 6 Freightliner truck with road load coefficients found here: 

https://www.nrel.gov/docs/fy17osti/64610.pdf

The vehicle parameters can be configured by the user, but shouldn't have to be. This includes the vehicle's mass, road inertia, road load coefficients and rated power.

The vehicle model also allows the user to input a grade schedule. The vehicle will respond to a given road grade as a function of distance and iterate through the schedule as if the vehicle were driving a loop. Details on grade will be given at its location in this notebook.




Note to self:
1. Is the 100ms loop working right when calculating deltaV?
2. Run the VM with 0 grade, 0 APP, and compare to actual coastdown results. It should be the exact coastdown.

## Requirements

The following are requirements for the vehicle model. Note that the cruise control module also has requirements which are listed in the Vehicle Model notebook.

### User Inputs

All user inputs are listed in the cell with the heading "#User-defined variables for CCM." The only exception is defining CAN bus parameters.





### Vehicle Speed

Vehicle speed is read from bus "CAN1" using the J1939 standard.

- Program will look for this message once every 100ms (meant for 10Hz data rate).
- arbitration ID 0x18FEF125
- parameter group number (PGN) 65265
- 2-byte message on bytes 6 and 7 (bytes 5 and 6 if counting from 0).
- multiplier: 1/256
- offset: 0
- units: [kph]



   
### Accelerator Pedal Position (APP)

Accelerator pedal position is transmitted on bus "CAN1" using the J1939 standard.

- Message is sent every 100ms (meant for 10Hz data rate)
- arbitration ID 0x18F00326
- parameter group number (PGN) 61444
- 1-byte message on byte 7 (2nd from last)
- multiplier: 1
- offset: -125
- units: [%]

Complete J1939 standard is available through SAE. These values taken from J1939 Digital Annex.


In [1]:
# Import libraries
import time
import math
import numpy as np
import socket
from socket import timeout
import struct
import sys

## User-Defined Variables and Vehicle Parameters

The only user-defined variable in the vehicle model is the vehicle's initial speed. Others can be considered constants.

How the vehicle parameters are used will be described later in the road load cell.

In [2]:
# User-defined variables for vehicle model
initial_kph = 80 #Need to enter this number into CCM kernel also

# These variables initialize the vehicle model and should not be changed
dist_prev = 0
dist = 0
prev_kph = initial_kph
kph = initial_kph
APP = 0 #Initial APP before receving value from from the CCM
checks = True  # Breaks proccess if errors occur
t_interval = .1  # Constant time interval [reflects CAN protocol wanted]

# Vehicle Parameters
mass = 11793  # Mass of vehicle [kg]
track_inertia = 1.03 * mass  # Estimated track inertia [kg]
# Constants in quadratic function for steady state road load
a = 579  # [N]
b = 0  # [N/kph]
c = 0.241512  # [N/kph^2]
ratedP = 179 # [kW]


# The following are not needed if running on a Beaglebone.
# These are used if running notebook on a PC that supports matplotlib and can graph the response
# Using to check results via graph
#graph = True
#x = [0.0]
#y = [initial_kph]
#x_value = 0.0
#y_value = initial_kph
#dist_diag = [0.0]




## Grade Schedule

A user-defined grade schedule is entered into the following cell. Two arrays are required: one for grade and one for distance. The index of each array must correspond with each other (i.e. the distance of the 5th number in the distance array corresponds to the grade in the 5th number of the grade array). Distance must only have a precision of 1 decimal point.

In [3]:
#Import grade profile
#These are random numbers to get module working, will import real grade schedule later
g = np.array([0, 0.5, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1]) #Unit is [%]
d = np.array([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2])
j_max = len(g)
k_max = len(d)
d_max = d[k_max-1] #Total distance of drive cycle
n = [0]
j = 0
k = 0
z = [0]


## APP Receive over CAN

The following cell is a Python function which reads APP from CAN1 and passes its value to the main loop of this notebook. Loop timing happens in this function: a CAN socket opens and listens for the correct message for up to 100ms. If the message is received, the function waits the remaining time within the 100ms window before continuing. If the message is not received, the function breaks, holds the previous speed value, and continues the main response loop. Therefore, the main response loop always runs at 10Hz. This function has no argument input and passes out APP as a floating-point integer.

More information on the Python socketcan can be found here:

https://python-can.readthedocs.io/en/master/interfaces/socketcan.html

In [4]:
#This cell simulates the ECM module receiving APP from the CAN bus
def CAN_APP_Rx():
    """Documentation: This function reads a single CAN message if it has ArbID 0x18F00326 for Driver engine demand
    (or APP)."""
    
    global APP
    
    APP_m = 1 #Multiplier for APP from J1939 standard [counts/%]
    APP_b = -125 #Offset for APP from J1939 [%]    
    
    # Open a socket and bind to it from SocketCAN
    sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)
    sock.setsockopt(socket.SOL_CAN_RAW, socket.CAN_RAW_ERR_FILTER, socket.CAN_ERR_MASK) #allows socket to receive error frames
    # The interface can be seen from the command prompt (ifconfig)
    # The can channel must be configured using the ip link commands
    interface = "can1"

    # Bind to the interface
    try:
        sock.bind((interface,))
    except OSError:
        print("Could not bind to interface '%s'\n" % interface)
        sys.exit()

    sock.settimeout(0.11) #testing this out
    # To match this data structure, the following struct format can be used:
    can_frame_format = "<LB3x8s"
    # Unsigned Long Integer (little endian), unsigned char (byte), three pad bytes, eight chars (bytes)
    # Note: this is 16 bytes


    # These are defined in can.h of the Linux kernel sources
    # e.g. https://github.com/torvalds/linux/blob/master/include/uapi/linux/can.h
    #print("socket.CAN_EFF_MASK = 0x{:08X}".format(socket.CAN_EFF_MASK))
    #print("socket.CAN_EFF_FLAG = 0x{:08X}".format(socket.CAN_EFF_FLAG))
    #print("socket.CAN_RTR_FLAG = 0x{:08X}".format(socket.CAN_RTR_FLAG))
    #print("socket.CAN_ERR_FLAG = 0x{:08X}".format(socket.CAN_ERR_FLAG))

    # Enter a loop to read and display the data
    t1, t2 = 0, 0
    
    while True:
        t1 = time.perf_counter()
        try:    
            can_packet = sock.recv(16) 
        except socket.timeout:
            APP = 0
            return APP
        can_id, can_dlc, can_data = struct.unpack(can_frame_format, can_packet)

        if (can_id == 2565866278): #ArbID for APP. The ArbID shows up as 98F00326. 18F00326 is actually sent
                                    #on the bus, not sure why it changes.  
            break

    extended_frame = bool(can_id & socket.CAN_EFF_FLAG)
    error_frame = bool(can_id & socket.CAN_ERR_FLAG)
    remote_tx_req_frame = bool(can_id & socket.CAN_RTR_FLAG)

    if error_frame:
        can_id &= socket.CAN_ERR_MASK
        can_id_string = "{:08X} (ERROR)".format(can_id)
    else: #Data Frame
        if extended_frame:
            can_id &= socket.CAN_EFF_MASK
            can_id_string = "{:08X}".format(can_id)
        else: #Standard Frame
            can_id &= socket.CAN_SFF_MASK
            can_id_string = "{:03X}".format(can_id)

    if remote_tx_req_frame:
        can_id_string = "{:08X} (RTR)".format(can_id)

    hex_data_print = ' '.join(["{:02X}".format(can_byte) for can_byte in can_data])
    #print("{:12.6f} {} [{}] {}".format(time.time(), can_id_string, can_dlc, hex_data_print))
    APP = int(hex_data_print[18:20],base=16)*APP_m + APP_b
    
    t2 = time.perf_counter()
    if (interval - (t2-t1)) > 0:
        time.sleep(interval - (t2-t1))
    
    return APP #units [%]


## APP send over CAN

The following cell is a Python function which takes in the value of vehicle speed, as calculated in the Vehicle Model function, and transmits it to CAN1. Vehicle speed is taken in as the argument of the function, and the function has no return value.

In [5]:
#This cell simulates the ECM transmitting vehicle speed onto the CAN bus
def CAN_kph_Tx(kph):
    """Documentation: This function takes in the value Vehicle Speed as a floating-point number (type='float'),
    turns it into the J1939 CAN message format, and sends the message on bus 'VCAN0' using SocketCAN.
    It uses PGN 65265, SPN 84. Arbitration is set to 18. These can be changed"""
    
    speed_m = 1/256 #Multiplier for speed from J1939 standard [counts/kph]
    speed_b = 0 #Offset for speed from J1939 [kph]

    kph_counts = int(kph / speed_m)
    kph_hex = hex(kph_counts)
    kph_byte1 = int(kph_hex[2] + kph_hex[3],16)
    kph_byte2 = int(kph_hex[4] + kph_hex[5],16)

    #print("kph: ",kph)
    #print("kph_counts: ",kph_counts)
    #print("kph_hex: ",kph_hex)
    #print("kph_byte1: ",kph_byte1)
    #print("kph_byte2: ",kph_byte2)
    
    # Open a socket and bind to it from SocketCAN
    sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)

    #Change this interface to match your desired connection
    interface = "can1"

    # Bind to the interface
    sock.bind((interface,))

    can_frame_format = "=lB3x8s"

    # Define the CAN Arbitration ID
    can_id = 0x18FEF125 #For Vehicle Speed on PGN 65265
    #Set the extended frame format bit.
    can_id |= socket.CAN_EFF_FLAG
    
    can_data = struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",kph_byte2) + struct.pack("B",kph_byte1) + struct.pack("B",0xFF)
    #The data length code cannot be larger than 8
    can_dlc = min(len(can_data),8)
    
    #Pack the information into 16 bytes
    can_packet = struct.pack(can_frame_format, can_id, can_dlc, can_data[:can_dlc] )
    #can_packet = struct.pack(can_frame_format, can_id, can_dlc, kph_Tx)

    #print(can_packet)
    
    #Send out the CAN frame
    sock.send(can_packet)

    pass

## Vehicle Model

This cell is the core of the vehicle model notebook. It is a python funtion with takes in APP and grade as arguments and passes speed as the output. The method used to determine vehicle road load is based on SAE J1263 “Road Load Measurement and Dynamometer Simulation Using Coastdown Techniques” and CFR Part 1066: commonly used for EPA compliance testing.

Vehicle road load can be considered an equivalent linear force resisting the forward motion of the vehicle. It is the sum of steady-state forces, force due to the acceleration of the vehicle's inertia, and a gravity component of the vehicle's mass based on road grade.

$$ RL_{Total} = RL_{SS} + RL_{inertia} + RL_{grade} $$

Steady-state road load is determined from coastdown procedures which yield a set of coefficients for a quadratic equation which models all constant-speed forces on the vehicle, such as windage, bearing losses and tire losses. These coefficients are included with vehicle parameters in a previous cell.

$$ RL_{SS} = A + B\cdot v + C\cdot v^2 $$

Road load due to the acceleration of the vehicle's inertia is simply the vehicle's road inertia times acceleration. Road inertia includes the vehicle's static mass and the rotational inertia of driveline components. Because directly measuring road inertia is extremely difficult, EPA allows for an estimate by taking the static weight of the vehicle and multiplying by 1.5% per axle (i.e. 3% for the two-axle box truck in this simulation) and is common industry practice.

$$ RL_{inertia} = J_{road}\cdot \frac{ds}{dt} $$

>where $\textit{s}$ = speed  and  $J_{road} = 1.03 \cdot mass$

Road load due to grade is a component of the vehicle's static weight based on road grade. The formula can be derived from a free-body diagram of a mass on an incline, but is beyond the scope of this document.

$$ RL_{grade} = m \cdot g \cdot sin(atan(grade)) $$
>where $m$ = mass in kg and $g$ = 9.81 $m/s^2$

After road load is determined, the model determines how much tractive effort the vehicle is producing. Tractive effort is the equivalent linear force pushing the vehicle forward by its powertrain and has units Newtons or pound-force. Tractive power is modeled as a simple linear scaling of APP and the engine's rated power from 0 to 100% and then converted to force using vehicle speed yielding tractive effort. The difference between road load and tractive effort is the force available to accelerate the inertia of the vehicle. This acceleration occurs over one loop, and an incremental speed change $\Delta v$ is calculated

$$ \Delta v = \frac{TractiveEffort - Road Load}{J_{road}} \cdot \Delta t $$

>where $\Delta t$ is a time increment of 0.1s.

More information on vehicle road load and dynamometer simulation can be found here:

https://www.ecfr.gov/cgi-bin/text-idx?SID=0ae8e4368fe34091e504e3a84830bc36&mc=true&node=pt40.37.1066&rgn=div5

In [6]:
# Vehicle Model
# This simulates how the vehicle speed physically changes based on a given APP.
# Road-load coefficients and inertia of a Class 6 Freightliner box truck are used
# (see earlier cell)
def vehicle_model(APP, grade):
    global kph
    global prev_kph

    accel = (kph - prev_kph) / interval
    roadLoad_SS = a + b * kph + c * kph ** 2 
    roadLoad_inertial = accel * track_inertia / 3600
    roadLoad_grade = mass * 9.81 * math.sin(math.radians(math.atan(grade / 100)))
    roadLoad_E = roadLoad_SS + roadLoad_inertial + roadLoad_grade

    # starting with => APP*rated power = tractive power [W] as estimate
    tractive_P = APP * ratedP / 100
    tractive_effort = tractive_P / kph * 3600
    delta_v = (tractive_effort - roadLoad_E) * interval / track_inertia
    kph = prev_kph + delta_v
    prev_kph = kph

    return kph



## Main Vehicle Model Loop

The following cell is the main response loop for the vehicle model. It simply calls the CAN-receive, vehicle model, and CAN-transmit functions in a loop while checking for errors each loop. 

It also iterates through the provided grade schedule and passes grade into the vehicle model function. This is done by dividing distance by maximum distance and looking up the remainder in the distance schedule. The corresponding grade is picked from the grade schedule and used for the next loop. This allows the distance/grade schedule to loop indefinitely, simulating a vehicle driving a circular route.

In [7]:
# Main Response Loop: Vehicle Model
global interval
t1 = time.perf_counter()
grade = g[0]
while (checks == True):

    CAN_APP_Rx()
    
    t2 = time.perf_counter()
    interval = t2-t1
    t1 = t2
    vehicle_model(APP,grade)
    
    CAN_kph_Tx(kph)
    
    
    # Distance & Grade
    # Description: 
    # dist_0 calculates incremental distance based on actual vehicle speed.
    # dist is rounded so that distance can be looked up in the grade schedule (i.e. boolean)
    # Because the grade schedule loops indefinitely, the % operator is used 
    # so it doesn't matter how many times the grade schedule has been looped through, or
    # how big distance gets. Grade will always be the same for a certain distance into any
    # given cycle.
    # index is the location within the grade/distance array where current distance matches.
    # That same index corresponds to a grade.
    # dist_diag is only used to help troubleshoot.
    # Grade has to be handled this way because vehicle speed changes based on user input
    # variables (Ki, Kp, etc...). So, grade has to be a function of distance, and not time.
    dist_float = dist_prev + kph * interval / 3600 #[km]
    dist_prev = dist_float
    dist = np.round(dist_float % d_max,decimals = 1)
    index = np.where(d == dist)
    dist_diag = z.append(dist)
    grade = g[index]
    
    #print("dist: ",dist)
    #print("dist_float: ",dist_float)
    #print("APP Rx: ", APP)
    #print("grade: ", grade)
    
    
    
# The following are not needed if running on a Beaglebone.
# These are used if running notebook on a PC that supports matplotlib and can graph the response
    # Generated Graph of Response
#    if graph == True:
#        x_value+=t_interval
#        y_value = kph
#        x.append(x_value)
#        y.append(y_value) 
#        print(x_value)#Just put in to make sure program is running properly
#        print(y_value) #Uncomment these three "print" statements to show numerical values
#        Use this stuff if running in a PC and can use matplotlib
#        if x_value >= 10000: #Change this number for how long you want loop to run
#            graph = False
#            checks = False
#            plt.plot(x,y)
#            plt.plot(x,y2)
#            plt.xlim(0,60)
#            plt.ylim(0,110)
#            plt.grid(True)
#            plt.xlabel('Time [s]')
#            plt.ylabel('Speed [kph]')
#            plt.title('Cruise Control Response')
#            plt.show()


IndexError: string index out of range

In [ ]:
kph